# Tu propio Generador de Textos con GPT-2

Ejercicio Propuesto, artículo completo en https://www.aprendemachinelearning.com/generacion-de-texto-en-espanol-con-gpt-2/

## Instalación del ambiente en Anaconda

In [ ]:
# conda create -n gpt2 python=3.9 -y
# Activa el nuevo ambiente con conda activate gpt2
# conda install numpy tqdm transformers
# si tienes GPU instala Pytorch con:
# conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia
# si no tienes GPU, instala con:
# conda install pytorch torchvision torchaudio cpuonly -c pytorch

In [2]:
import os
import time
import datetime

import numpy as np
import random
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AdamW, get_linear_schedule_with_warmup

In [13]:
# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)

if torch.cuda.is_available():
    print("Usar GPU")
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    # Tell pytorch to run this model on the GPU.
    device = torch.device("cuda")
    batch_size = 3

else:
    print("usar CPU")
    device = torch.device("cpu")
    batch_size = 1


Usar GPU


In [4]:
# Load the GPT tokenizer.

tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt-2-spanish", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

model = AutoModelForCausalLM.from_pretrained("flax-community/gpt-2-spanish")

control_code = "ibai"

special_tokens_dict = {
         "additional_special_tokens": ['f"<|{control_code}|>"'],
}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
unk_tok_emb = model.transformer.wte.weight.data[tokenizer.unk_token_id, :]
for i in range(num_added_toks):
        model.transformer.wte.weight.data[-(i+1), :] = unk_tok_emb


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
class GPT2Dataset(Dataset):

  def __init__(self, control_code, tokenizer, archivo_texto = 'all.txt', max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    print('loading text...')
    sentences = open(archivo_texto, 'r', encoding="utf-8").read().lower().split('\n')
    print('qty:',len(sentences))

    for row in tqdm(sentences):
      encodings_dict = tokenizer('<|startoftext|>'+ f"<|{control_code}|>" + row + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")
      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

In [12]:
dataset = GPT2Dataset(control_code, tokenizer, archivo_texto="ibai_textos.txt", max_length=768)

# Split into training and validation sets
train_size = int(0.99 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

loading text...
qty: 10293


100%|██████████| 10293/10293 [00:03<00:00, 3186.97it/s]

10,190 training samples
  103 validation samples


In [9]:
# some parameters to train
epochs = 1
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8
# this produces sample output every x steps
sample_every = 500
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

# Entrenar el modelo GPT-2 con nuestro Dataset

In [19]:
total_t0 = time.time()

model = model.to(device)

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    
    t0 = time.time()

    total_eval_loss = 0
    nb_eval_steps = 0

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
#Average training loss: 0.28
#Training epoch took: 1:23:32 mode_save2 va bastante bien, solo 1 epoch 



======== Epoch 1 / 1 ========
Training...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   500  of  3,397. Loss: 0.16669423878192902.   Elapsed: 0:08:07.
0:  ladrones de la calle. También por el otro lado, que e por por este canal de youtube. por por los cuatro hasta aquí. por este canal. el otro. 
I-B-D. ¿ cuál es la razón por la que crees que un canal se va a llevar una sorpresa cuando entre los cuatro, ¿ has visto algo nuevo en el cine, por ejemplo, y piensas que lo va a hacer una sola persona con un equipo de sonido, pantalla, efectos especiales, etc.? ¿ has visto algún video o alguna serie que no haya sido grabada como tal? ¿ estás en contra de la idea, por favor? ¿ de que alguien se vaya de vacaciones con una silla de ruedas? ¿ te gusta el cine? ¿ cuánto dinero y


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 1,000  of  3,397. Loss: 0.24639984965324402.   Elapsed: 0:16:11.
0:  significa<|ibai|> ¡vale, pero qué hijo de puta! ¡vale, vale, vale! ¡vale, vale, vale! ¡vale, vale, vale! ¡vale, vale, vale! ¡vale, vale, vale! ¡vale, vale, vale! ¿ qué cojones es esto? ¡no, no! ¡no, no! ¡no, no! ¡que esto tiene la polla! ¡y con dos cojones! ¡sí! ¡vale, vale! ¡vale, vale! ¡vale, vale! ¡vale, vale! ¡vale, vale! ¡vale, vale! ¡vale, vale! ¡vale! ¡vale, vale! ¡vale, vale! ¡vale, vale! ¡vale! ¡vale, vale! ¡vale, vale! ¡vale, vale! ¡vale, vale! ¡vale, vale! ¡vale, vale! ¡vale, vale! ¡vale, vale! ¡vale, vale!


...

  Batch 2,000  of  3,397. Loss: 0.24323517084121704.   Elapsed: 0:31:37.
0:  decirme<|ibai|> no sé.  pero si la tengo.  ¡jajaja! me quedo por mi,


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 2,500  of  3,397. Loss: 0.19146695733070374.   Elapsed: 0:39:20.
0:  discos<|ibai|> claro, la gente te va a decir.  el tema de que me lo puedo hacer solo.  que me lo puedo hacer.  ya de repente, se me acaba el dinero. 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 3,000  of  3,397. Loss: 0.25842446088790894.   Elapsed: 0:47:03.
0: dete<|ibai|> a ver, a ver, mira, lo que sí me he traído ahora es lo siguiente, como me decía uno del barça, me llevo a una peña, que son los creadores del tal tato, que es el que me está yendo a vivir, los artistas del tal tal.  porque no ha sido mi peor fichaje, es más, a mí me viene bien la fama y la fama, porque tengo un artista que es tremendo en lo personal, me queda mucho por delante y quiero que sea un poco como lo he sido él.  pues no lo puedo permitir. 

  Average training loss: 0.23
  Training epoch took: 0:53:10
  Validation took: 0:00:00

Training complete!
Total training took 0:53:10 (h:mm:ss)


# Guardar el modelo Entrenado

In [20]:
# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
output_dir = './model_gpt_ibai/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


Saving model to ./model_save3/


('./model_save3/tokenizer_config.json',
 './model_save3/special_tokens_map.json',
 './model_save3/vocab.json',
 './model_save3/merges.txt',
 './model_save3/added_tokens.json',
 './model_save3/tokenizer.json')

# Cargar el modelo entrenado, para Inferencia

In [16]:
# Load a trained model and vocabulary that you have fine-tuned
#output_dir = './model_gpt_ibai'
#model = GPT2LMHeadModel.from_pretrained(output_dir)
#tokenizer = AutoTokenizer.from_pretrained(output_dir, bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
#model.to(device)

c:\Users\nacho\anaconda3\envs\myenv\lib\site-packages\torch\cuda\__init__.py:125: UserWarning: 
NVIDIA GeForce RTX 3080 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA GeForce RTX 3080 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dro

# Generación de Texto con el nuevo Modelo

In [21]:
model.eval()

prompt = "<|startoftext|>" + "<|"+control_code+"|>" + "¿ qué es el fútbol ?"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
#print(generated)

sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=8
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[50256,    32,    96, 17739,    77,    96,    34,  1699,  1245,   225,
            73,    87,   291,  4551,  7381]], device='cuda:0')
0: <|ibai|>¿ qué es el fútbol? ¿ qué es el fútbol? porque yo creo que son los mejores momentos de la vida del mundo.  eh no sé, yo creo que son los mejores momentos de la vida del mundo.  eh y eso es lo que me llama la atención.  eh  ¡gracias! gracias. 


1: <|ibai|>¿ qué es el fútbol? no, la primera.  eh  pero hay que hacer algo.  ¿ quién es? de la mesa en donde estaba el tal.  se fue a la concha de tu madre. 


2: <|ibai|>¿ qué es el fútbol? pero no me lo he pillado.  ah, pero también es una cosa totalmente personal.  eh  pues, creo que, bueno, a ver si  te da tiempo a jugar lo que quieras.  me ha dado un tiempo de juego ¿ eh? creo que es un poco complicado. 


3: <|ibai|>¿ qué es el fútbol? fútbol es la suma de sus defectos, del mal que uno puede poner en uno, de un mal que tú quieres que la gente haga como si nada, es decir, yo soy el contrar